In [10]:
import json
import random
import warnings
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adagrad
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import class_weight
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
train_path = 'Data/train.json'
dev_path = 'Data/validation.json'
test_path = 'Data/test.json'
device = 'cuda'

with open(train_path, 'r') as f:
    train = json.load(f)
with open(dev_path, 'r') as f:
    dev = json.load(f)
with open(test_path, 'r') as f:
    test = json.load(f)

In [ ]:
model_name = "dbmdz/bert-base-turkish-128k-uncased"
# model_name = "bert-base-multilingual-cased"
# model_name = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

In [ ]:
mapping = {'negative':0, 'neutral':1, 'positive':2}

def to_id(text):
    return torch.tensor(tokenizer.encode(text))

def feature_extraction(data, max_seq_length=50):
    sentences = [el["sentence"] for el in data]
    y = [mapping[el["value"]] for el in data]
    
    return tokenizer(sentences, padding='max_length', truncation=True, max_length=max_seq_length), y

In [ ]:
class BOUNDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
categories = [0, 1, 2]

x_train, y_train = feature_extraction(train)
train_set = BOUNDataset(x_train, y_train)
x, y = feature_extraction(dev)
dev_set = BOUNDataset(x, y)
x, y = feature_extraction(test)
test_set = BOUNDataset(x, y)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
weights = torch.Tensor(compute_class_weight(classes=[0,1,2], y=y_train, class_weight="balanced")).to(device)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


from transformers import Trainer, TrainingArguments

class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels)
        return (loss, outputs) if return_outputs else loss


training_args = TrainingArguments(
    output_dir='transformer_results',
    num_train_epochs=10,
    per_device_train_batch_size=24,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=36,
    warmup_steps=300,
    weight_decay=0.1,
    learning_rate=1e-5,
    logging_dir='transformer_logs',
    logging_steps=5,
    load_best_model_at_end=True,
    evaluation_strategy = "epoch",
    metric_for_best_model = "recall",
    do_eval=True,
    save_total_limit=1)

trainer = MultilabelTrainer(
    args=training_args,
    train_dataset=train_set,
    eval_dataset=dev_set,
    compute_metrics=compute_metrics,
    model_init=model_init
)

In [ ]:
trainer.train() 
# if you want to use already finetuned model, use the command below instead
# trainer.model = trainer.model.from_pretrained("berturk").to(device) 

In [ ]:
trainer.evaluate(dev_set)

In [ ]:
trainer.evaluate(test_set)